# Data preprocessing

In [1]:
# Import libraries
import pandas as pd
import os
import billboard
import lyricsgenius
import json
import re

In [2]:
# Loading data and adding year columns and wrapping up billboard songs (1950-2015)
cwd = os.getcwd()
data_path = os.chdir(cwd + "\data")
all_csv_files = os.listdir(data_path)
songs_50_15 = []

for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0)
    year = os.path.basename(filename)[:-4]
    songs["Year"] = year
    songs_50_15.append(songs)

In [3]:
# Downloading data from years 2016-2020 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date = None, year = y, fetch = True, timeout = 25)
    songs_billboard.append(songs)

In [4]:
# Preprocessing data from years 2016-2020
song_titles = []
artists = []
for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(" by ")
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared')

# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))

# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dict_ = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dict_)
        songs.append(as_pandas_df)
    return songs

songs_16_20 = dicts_of_dates(dates)

all_songs = songs_50_15 + songs_16_20
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.drop('jPosition', inplace = True, axis = 1) # drop 'jPosition' column
all_time_billboard_wrap_up.reset_index(drop = True, inplace = True) # reset indexing
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950
1,2,Nat King Cole,Mona Lisa,1950
2,3,Anton Karas,Third Man Theme,1950
3,4,Gary and Bing Crosby,Sam's Song,1950
4,5,Gary and Bing Crosby,Simple Melody,1950
...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020
6676,97,Luke Combs,Lovin' On You,2020
6677,98,Moneybagg Yo,Said Sum,2020
6678,99,H.E.R. Featuring YG,Slide,2020


# Spotify tokens

# Genius lyrics

In [5]:
# Spliting authors by 'and' and 'Featuring'
artists = list(all_time_billboard_wrap_up.loc[:,"Artist"])
split_artists = []
for artist in artists:
    if any(re.findall(r'and|Featuring', str(artist))):
        result = re.split(r'and|Featuring', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up["Split names"] = split_artists
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,Split names
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole
2,3,Anton Karas,Third Man Theme,1950,Anton Karas
3,4,Gary and Bing Crosby,Sam's Song,1950,Gary
4,5,Gary and Bing Crosby,Simple Melody,1950,Gary
...,...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020,Morgan Wallen
6676,97,Luke Combs,Lovin' On You,2020,Luke Combs
6677,98,Moneybagg Yo,Said Sum,2020,Moneybagg Yo
6678,99,H.E.R. Featuring YG,Slide,2020,H.E.R.


In [23]:
# Lyrics to jsons
genius_token = os.environ.get('genius_token')
genius = lyricsgenius.Genius(genius_token)
songs = list(all_time_billboard_wrap_up.loc[:, "Song Title"])
artists = list(all_time_billboard_wrap_up.loc[:,"Split names"])

def return_lyrics_json(song, artist, json_name):
    song_lyrics = genius.search_song(song, artist)
    with open(json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': song_lyrics.lyrics}, f, ensure_ascii = False, indent = 4)

# function for no lyrics
# function for searchig error

for i in range(len(artists)):
    try:
        return_lyrics_json(songs[i], artists[i], str(i)+'.json')
    except AttributeError:
        pass

Searching for "Goodnight Irene" by Gordon Jenkins...
Done.
Searching for "Mona Lisa" by Nat King Cole...
Done.
Searching for "Third Man Theme" by Anton Karas...
Done.
Searching for "Sam's Song" by Gary...
Done.
Searching for "Simple Melody" by Gary...
Done.
Searching for "Music, Music, Music" by Teresa Brewer...
Done.
Searching for "Third Man Theme" by Guy Lombardo...
Done.
Searching for "Chattanoogie Shoe Shine Boy" by Red Foley...
Done.
Searching for "Harbor Lights" by Sammy Kaye...
Done.
Searching for "It Isn't Fair" by Sammy Kaye...
Done.
Searching for "If I Knew You Were Coming I'd've Baked A Cake" by Eileen Barton...
Done.
Searching for "Bonaparte's Retreat" by Kay Starr...
Done.
Searching for "Tzena, Tzena, Tzena" by Gordon Jenkins...
No results found for: 'Tzena, Tzena, Tzena Gordon Jenkins'
Searching for "There's No Tomorrow" by Tony Martin...
Done.
Searching for "The Thing" by Phil Harris...
Done.
Searching for "Sentimental Me" by Ames Brothers...
Done.
Searching for "I Wanna

Done.
Searching for "The Happy Wanderer" by Frank Weir...
Done.
Searching for "Answer Me My Love" by Nat King Cole...
Done.
Searching for "Stranger In Paradise" by Tony Bennett...
Done.
Searching for "If I Give My Heart To You" by Doris Day...
Done.
Searching for "If You Love Me (Really Love Me)" by Kay Starr...
Done.
Searching for "Skokiaan" by Ralph Marterie...
No results found for: 'Skokiaan Ralph Marterie'
Searching for "Hold My Hand" by Don Cornell...
Done.
Searching for "Changing Partners" by Patti Page...
Done.
Searching for "Papa Loves Mambo" by Perry Como...
Done.
Searching for "Shake, Rattle And Roll" by Bill Haley...
Done.
Searching for "Rags To Riches" by Tony Bennett...
Done.
Searching for "In The Chapel In The Moonlight" by Kitty Kallen...
Done.
Searching for "Stranger In Paradise" by Four Aces...
Done.
Searching for "Here" by Tony Martin...
Done.
Searching for "Cherry Pink And Apple Blossom White" by Perez Prado...
Done.
Searching for "Rock Around The Clock" by Bill Hale

Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)